In [1]:
def get_time(minute_after_midnight):
    # hh:mm
    hh = minute_after_midnight // 60
    mm = minute_after_midnight % 60
    return str(hh) + ":" + str(mm).zfill(2)


In [2]:
import import_ipynb
from dijkstra import nodes, arc_matrix, get_shortest_path, format_path

importing Jupyter notebook from dijkstra.ipynb
('1', '2', '3', '4', '5n', '5s', '6', '7', '8', '9', '10', '10b', '11n', '11s', '12', '13', '14', '15', '16', '17', '19', '20', '20b', '21', '22', '23', '24', '25/1', '25/2', '26', '31', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22')
[[inf inf  7. ... inf inf inf]
 [inf inf inf ... inf inf inf]
 [ 7. inf inf ... inf inf inf]
 ...
 [inf inf inf ... inf  4. inf]
 [inf inf inf ...  4. inf inf]
 [inf inf inf ... inf inf inf]]
X19 -> X18 -> 23
7.0
21: 0
X22: 2.0
20b: 3.0
20: 8.0
X21: 15.0
X20: 19.0
25/1: 20.0
19: 22.0
25/2: 22.0
26: 22.0
X19: 24.0
22: 26.0
X18: 28.0
23: 31.0
X17: 31.0
24: 32.0
X16: 32.0
31: 33.0
7: 35.0
17: 35.0
X15: 35.0
9: 37.0
12: 38.0
X14: 38.0
11s: 39.0
X2: 39.0
6: 40.0
8: 40.0
X7: 40.0
X3: 41.0
X8: 41.0
10: 42.0
X4: 43.0
X9: 43.0
X13: 43.0
5s: 44.0
X1: 44.0
X5: 44.0
X10: 44.0
5n: 45.0
11n: 45.0
X12: 45.0
4: 47.0
14: 47.0
15: 

In [3]:
import random
import numpy as np

random.seed(0)

max_minutes = 24*60

distribuzione_tamponi_al_minuto = np.array([random.randint(0, 10) for _ in range(max_minutes)])

distribuzione_reparti = np.array([random.random() for _ in range(len(nodes))])
junction_nodes_start_index = nodes.index('X1')
distribuzione_reparti[junction_nodes_start_index:] = 0
# distribuzione_reparti /= distribuzione_reparti.sum()
# print(distribuzione_reparti)

print(distribuzione_tamponi_al_minuto)
print(distribuzione_reparti)

[6 6 0 ... 3 0 5]
[0.5290949  0.94135742 0.68025796 0.630908   0.62781515 0.49698971
 0.73091927 0.24919444 0.89175426 0.27447266 0.94494501 0.92649671
 0.07792452 0.4481797  0.74403628 0.44965407 0.50889902 0.80682394
 0.70499216 0.95800422 0.16448599 0.92355929 0.92798625 0.63474894
 0.94039083 0.25268559 0.88178728 0.77347929 0.609689   0.09062924
 0.03013435 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.        ]


In [4]:
def aggiornamento_tamponi(tamponi_per_reparto, current_minute):
    return aggiornamento_tamponi_semplice(tamponi_per_reparto, current_minute)

def aggiornamento_tamponi_semplice(tamponi_per_reparto, current_minute):
    if(current_minute==1):
        reparto = nodes.index('9')
        tamponi_per_reparto[reparto] += 10
        
    return tamponi_per_reparto

def aggiornamento_tamponi_statistica(tamponi_per_reparto, current_minute):
    nuovi_tamponi_per_reparto = (distribuzione_tamponi_al_minuto[current_minute] * distribuzione_reparti).astype(int)
    tamponi_per_reparto += nuovi_tamponi_per_reparto

    return tamponi_per_reparto

In [5]:
def pianifica(tamponi_per_reparto, current_position, current_minute):
    return pianifica_gotomax(tamponi_per_reparto, current_position, current_minute)

def pianifica_gotomax(tamponi_per_reparto, start, current_minute):
    end = np.argmax(tamponi_per_reparto)
    
    if tamponi_per_reparto[end] == 0: # non ci sono tamponi da raccogliere
        return None

    (path, distance) = get_shortest_path(arc_matrix, start, end)
    # print(format_path(path))
    # print(distance)

    path = path[1:]
    return path

In [36]:
def move_toward(current_position, next_position, moving_time):
    distance = int(get_shortest_path(arc_matrix, current_position, next_position)[1])
    moving_time = min(moving_time+1, distance)
    print(f"Moving from {nodes[current_position]} to {nodes[next_position]} ... {moving_time}/{distance}")
    return distance, moving_time

In [7]:
def reconsider():
    return False

In [47]:
tamponi_per_reparto = np.zeros(len(nodes)).astype(int)
current_position = nodes.index('X1')
current_minute = 0
moving_time = 0
tamponi_trasportati = 0

while current_minute < max_minutes:
    print(get_time(current_minute), nodes[current_position])
        
    tamponi_per_reparto = aggiornamento_tamponi(tamponi_per_reparto, current_minute)
    print(tamponi_per_reparto)

    piano = pianifica(tamponi_per_reparto, current_position, current_minute)
    print(format_path(piano))

    while piano not in [[], None]:
        next_position = piano[0]
        distance, moving_time = move_toward(current_position, next_position, moving_time)
        current_minute += 1
        if distance == moving_time: # arrivato a destinazione
            print(f"Arrived at {nodes[next_position]}")
            current_position = next_position
            piano = piano[1:]
            moving_time = 0
            
            if tamponi_per_reparto[current_position] > 0:
                tamponi_trasportati += tamponi_per_reparto[current_position]
                tamponi_per_reparto[current_position] = 0
                print(f"Tamponi trasportati: {tamponi_trasportati}")
        
        
        print()
        print(get_time(current_minute), nodes[current_position])
        
        tamponi_per_reparto = aggiornamento_tamponi(tamponi_per_reparto, current_minute)
        print(tamponi_per_reparto)

        if reconsider():
            piano = pianifica(tamponi_per_reparto, current_position, current_minute)
        
        print(format_path(piano))
        
        # TESTING
        if current_minute > 10:
            break
    
    # TESTING
    if current_minute > 10:
        break

    current_minute += 1
    print()

0:00 X1
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
No path

0:01 X1
[ 0  0  0  0  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0]
X2 -> 9
Moving from X1 to X2 ... 1/5

0:02 X1
[ 0  0  0  0  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0]
X2 -> 9
Moving from X1 to X2 ... 2/5

0:03 X1
[ 0  0  0  0  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0]
X2 -> 9
Moving from X1 to X2 ... 3/5

0:04 X1
[ 0  0  0  0  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0]
X2 -> 9
Moving from X1 to X2 ... 4/5

0:05 X1
[ 0  0  0  0  0  0  0  0  0 10